In [0]:
%pip install --upgrade langgraph langchain_core databricks-langchain mlflow-skinny[databricks]

In [0]:
import mlflow
mlflow.langchain.autolog()

In [0]:
from databricks_langchain.chat_models import ChatDatabricks

llm = ChatDatabricks(
    endpoint="databricks-llama-4-maverick"  # Just the endpoint name
)


In [0]:
from langgraph.graph import Graph

graph = Graph()

def user_node(state):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

graph.add_node("user_node", user_node)
graph.set_entry_point("user_node")
graph.set_finish_point("user_node")

compiled_graph = graph.compile()
output = compiled_graph.invoke({"messages": [HumanMessage(content="Hello")]})

Register & Deploy Under Unity Catalog

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")

In [0]:
from langchain_core.messages import HumanMessage

# Use to_dict() to avoid Pydantic/LangChain objects
input_example = {
    "messages": [HumanMessage(content="Hello!").dict()]
}


In [0]:
output = graph.invoke({
    "messages": [HumanMessage(content="Hello!")]
})



In [0]:
import json

# Ensure both input and output are JSON serializable
json.dumps(input_example)
output = json.dumps(output, default=lambda o: o.to_dict() if hasattr(o, "to_dict") else str(o))


In [0]:
from mlflow.models.signature import infer_signature

signature = infer_signature(input_example, output)


In [0]:
from langchain_core.runnables import RunnableLambda

wrapped_graph = RunnableLambda(lambda x: compiled_graph.invoke(x))

In [0]:
import mlflow

with mlflow.start_run():
    mlflow.langchain.log_model(
        lc_model=wrapped_graph,
        name="langgraph_flow",
        registered_model_name="genai.default.my_langgraph_model",
        input_example=input_example,
        signature=signature  # Optional but ideal
    )
